In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import torch.nn.functional as F

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_dim, external_dim):  # external_dim为外部变量的维度
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            #nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            #nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            #nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            #nn.Dropout(0.2),
        )
        self.external_dim = external_dim
        
        # 新增3层DNN
        self.additional_layers = nn.Sequential(
            nn.Linear(64 + external_dim, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            #nn.Dropout(0.2),
            
            
        )
        
        self.fc = nn.Linear(32, 1)  # 修改后的全连接层
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x_main, external_var):
        # 前向传播到 nn.Linear(64, 32) 层
        x_main = self.model(x_main)
        
        # 在这里将外部变量与 x_main 拼接
        x = torch.cat([x_main, external_var], dim=1)
        
        # 新增3层DNN的前向传播
        x = self.additional_layers(x)
        
        # 通过最终的全连接层得到输出
        x = self.fc(x).squeeze(-1)
        
        return x